In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [5]:
df = pd.read_csv('data/coursera_courses.csv')

In [6]:
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [7]:
df.shape

(3522, 7)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [9]:
df.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [10]:
df.columns

Index(['Course Name', 'University', 'Difficulty Level', 'Course Rating',
       'Course URL', 'Course Description', 'Skills'],
      dtype='object')

In [11]:
df = df[['Course Name', 'Difficulty Level', 'Course URL', 'Course Description', 'Skills']]

In [12]:
df.head()

,Course Name,Difficulty Level,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [13]:
df['Course Name'] = df['Course Name'].str.replace(' ', ',')
df['Course Name'] = df['Course Name'].str.replace(':', '')
df['Course Name'] = df['Course Name'].str.replace(',,', ',')

df['Course Description'] = df['Course Description'].str.replace(' ', ',')
df['Course Description'] = df['Course Description'].str.replace(',,', ',')
df['Course Description'] = df['Course Description'].str.replace(':', '')
df['Course Description'] = df['Course Description'].str.replace('_', '')
df['Course Description'] = df['Course Description'].str.replace('(', '')
df['Course Description'] = df['Course Description'].str.replace(')', '')

df['Skills'] = df['Skills'].str.replace('(', '')
df['Skills'] = df['Skills'].str.replace(')', '')

In [14]:
df.head()

,Course Name,Difficulty Level,Course URL,Course Description,Skills
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...",Beginner,https://www.coursera.org/learn/write-a-feature...,"Write,a,Full,Length,Feature,Film,Script,In,thi...",Drama Comedy peering screenwriting film D...
1,"Business,Strategy,Business,Model,Canvas,Analys...",Beginner,https://www.coursera.org/learn/canvas-analysis...,"By,the,end,of,this,guided,project,you,will,be,...",Finance business plan persona user experienc...
2,"Silicon,Thin,Film,Solar,Cells",Advanced,https://www.coursera.org/learn/silicon-thin-fi...,"This,course,consists,of,a,general,presentation...",chemistry physics Solar Energy film lambda...
3,"Finance,for,Managers",Intermediate,https://www.coursera.org/learn/operational-fin...,"When,it,comes,to,numbers,there,is,always,more,...",accounts receivable dupont analysis analysis...
4,"Retrieve,Data,using,Single-Table,SQL,Queries",Beginner,https://www.coursera.org/learn/single-table-sq...,"In,this,course,you�ll,learn,how,to,effectively...",Data Analysis select sql database management...


In [15]:
df.columns

Index(['Course Name', 'Difficulty Level', 'Course URL', 'Course Description',
       'Skills'],
      dtype='object')

In [16]:
df['tags'] = df['Course Name'] + df['Difficulty Level'] + df['Course Description'] + df['Skills']

In [17]:
df['tags'].iloc[1]

'Business,Strategy,Business,Model,Canvas,Analysis,with,MiroBeginnerBy,the,end,of,this,guided,project,you,will,be,fluent,in,identifying,and,creating,Business,Model,Canvas,solutions,based,on,previous,high-level,analyses,and,research,data.,This,will,enable,you,to,identify,and,map,the,elements,required,for,new,products,and,services.,Furthermore,it,is,essential,for,generating,positive,results,for,your,business,venture.,This,guided,project,is,designed,to,engage,and,harness,your,visionary,and,exploratory,abilities.,You,will,use,proven,models,in,strategy,and,product,development,with,the,Miro,platform,to,explore,and,analyse,your,business,propositions.,,We,will,practice,critically,examining,results,from,previous,analysis,and,research,results,in,deriving,the,values,for,each,of,the,business,model,sections.Finance  business plan  persona user experience  business model canvas  Planning  Business  project  Product Development  presentation  Strategy business business-strategy'

In [18]:
new_df = df[['Course Name', 'Course URL', 'tags']]

In [19]:
new_df["Course Name"] = df['Course Name'].str.replace(',', ' ')

C:\Users\ghole\AppData\Local\Temp\ipykernel_26380\1286306147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Course Name"] = df['Course Name'].str.replace(',', ' ')


In [20]:
new_df.rename(columns = {'Course Name': 'course_name', 'Course URL': 'course_url'}, inplace = True)

C:\Users\ghole\AppData\Local\Temp\ipykernel_26380\4208862616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns = {'Course Name': 'course_name', 'Course URL': 'course_url'}, inplace = True)


In [21]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\ghole\AppData\Local\Temp\ipykernel_26380\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [22]:
new_df.head()

,course_name,course_url,tags
0,Write A Feature Length Screenplay For Film Or ...,https://www.coursera.org/learn/write-a-feature...,"write,a,feature,length,screenplay,for,film,or,..."
1,Business Strategy Business Model Canvas Analys...,https://www.coursera.org/learn/canvas-analysis...,"business,strategy,business,model,canvas,analys..."
2,Silicon Thin Film Solar Cells,https://www.coursera.org/learn/silicon-thin-fi...,"silicon,thin,film,solar,cellsadvancedthis,cour..."
3,Finance for Managers,https://www.coursera.org/learn/operational-fin...,"finance,for,managersintermediatewhen,it,comes,..."
4,Retrieve Data using Single-Table SQL Queries,https://www.coursera.org/learn/single-table-sq...,"retrieve,data,using,single-table,sql,queriesbe..."


In [23]:
new_df.shape

(3522, 3)

In [24]:
cv = CountVectorizer(max_features=5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [25]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
ps = PorterStemmer()

In [27]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [28]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\ghole\AppData\Local\Temp\ipykernel_26380\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [29]:
similarity = cosine_similarity(vectors)

In [30]:
def recommend(course):
    course_index = new_df[new_df['course_name'] == course].index[0]
    distance = similarity[course_index]
    course_list = sorted(list(enumerate(distance)), reverse = True, key = lambda x:x[1])[1:7]

    recommended_courses = []
    for i in course_list:
        course_name = new_df.iloc[i[0]].course_name
        course_url = new_df.iloc[i[0]].course_url
        recommended_courses.append({'name': course_name, 'url': course_url})
    return recommended_courses

In [31]:
new_df.head()

,course_name,course_url,tags
0,Write A Feature Length Screenplay For Film Or ...,https://www.coursera.org/learn/write-a-feature...,"write,a,feature,length,screenplay,for,film,or,..."
1,Business Strategy Business Model Canvas Analys...,https://www.coursera.org/learn/canvas-analysis...,"business,strategy,business,model,canvas,analys..."
2,Silicon Thin Film Solar Cells,https://www.coursera.org/learn/silicon-thin-fi...,"silicon,thin,film,solar,cellsadvancedthis,cour..."
3,Finance for Managers,https://www.coursera.org/learn/operational-fin...,"finance,for,managersintermediatewhen,it,comes,..."
4,Retrieve Data using Single-Table SQL Queries,https://www.coursera.org/learn/single-table-sq...,"retrieve,data,using,single-table,sql,queriesbe..."


In [32]:
recommend("Retrieve Data using Single-Table SQL Queries")

[{'name': 'Creating Database Tables with SQL',
  'url': 'https://www.coursera.org/learn/creating-database-tables-with-sql'},
 {'name': 'Manipulating Data with SQL',
  'url': 'https://www.coursera.org/learn/manipulating-data-with-sql'},
 {'name': 'Create Relational Database Tables Using SQLiteStudio',
  'url': 'https://www.coursera.org/learn/create-relational-database-table-sqlitestudio'},
 {'name': 'Retrieve Data with Multiple-Table SQL Queries',
  'url': 'https://www.coursera.org/learn/multiple-table-sql-queries'},
 {'name': 'Advanced SQL Retrieval Queries in SQLiteStudio',
  'url': 'https://www.coursera.org/learn/Advanced-sql-retrieval-queries-in-sqlitestudio'},
 {'name': 'Relational Database Support for Data Warehouses',
  'url': 'https://www.coursera.org/learn/dwrelational'}]

In [33]:
pickle.dump(similarity, open("models/similarity.pkl", "wb"))
pickle.dump(new_df[['course_name', 'course_url']], open("models/courses.pkl", "wb"))
pickle.dump(new_df[['course_name', 'course_url']].to_dict('records'), open("models/course_list.pkl", "wb"))